STAGE 1
# DATA PREPARATION

DOWNLOAD DATASET
DATA PREPROCESSING
CREATE DATALOADERS


STAGE 2
# MODEL SETUP
INITIALIZE THE MODEL
LOAD PRETRAINED MODEL WEIGHTS OF GPT 2
MODIFY THE FINAL OUTPUT LAYERS OF MODEL FOR FINETUNING
IMPLEMENT THE EVALUATION UTILITIES


STAGE 3
# MODEL FINETUNING AND USAGE
FINETUNE MODEL
EVALUATE THE MODEL >ACCURACY , LOSS .
TEST THE MODEL ON NEW DATA

# stage 1 begning data preprocessing

In [45]:
import pandas as pd

In [ ]:
df = pd.read_csv("./original_dataset/SMSSpamCollection.csv",sep= ",", header= None, names=["Label", "Text"])
df.head()

,Label,Text
0,label,text
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...


In [47]:
print("The description of the dataset \n",df.describe())
print(" -------------------------------")
print("The shape of the dataset",df.shape)
print(" -------------------------------")
print("The value counts of the dataset\n",df["Label"].value_counts())


The description of the dataset 
        Label                    Text
count   5573                    5573
unique     3                    5170
top      ham  Sorry, I'll call later
freq    4825                      30
 -------------------------------
The shape of the dataset (5573, 2)
 -------------------------------
The value counts of the dataset
 Label
ham      4825
spam      747
label       1
Name: count, dtype: int64


we need to create a balanced dataset you can see ham has 4825 samples and spam has 747 samples, its obviosly imbalanced dataset , we will create a new balanced dataset form this . We will take 747 sample from each class and create a new dataset.

In [90]:
def create_balanced_dataset(df):
    
    #count the frequncy of spam and extract random samples of ham from dataset , such that both ham and spam sample count or you can say frquency is same
    num_spam = df[df['Label'] == "spam"].shape[0]
    ham_instances = df[df['Label'] == "ham"].sample(num_spam, random_state=42) #it will extract 747(num_spam) random samples of ham.
    spam_instances = df[df['Label'] == "spam"]
    balanced_df = pd.concat([ham_instances,spam_instances])
    return balanced_df


In [91]:
#main
balanced_df =  create_balanced_dataset(df)
balanced_df['Label'].value_counts()

Label
ham     747
spam    747
Name: count, dtype: int64

# saving the balanced dataset into a seprate csv file 
we will do training and validation , testing and finetuing with the help of this dataset

In [ ]:
# balanced_df.to_csv("./balanced_dataset_prepared/balanced_dataset.csv", index = False)

In [72]:
balanced_df['Label'].value_counts()

Label
ham     747
spam    747
Name: count, dtype: int64

# we have balanced dataframe prepared already so we will use it and do  Label encoding on it. so that our model understand the numbers better.
NOTE: we are just making changes on dataframe and not on original balanced dataset

# Label Encoding of Labels['ham','spam']
ham -> 0
spam -> 1

In [ ]:
balanced_df['Label'] = balanced_df['Label'].map({'ham':0, 'spam':1})

you can see that we have encoded the labels spam as 1 and ham as 0

In [ ]:
print(balanced_df['Label'].value_counts())

Label
0    747
1    747
Name: count, dtype: int64


# We will split the dataset into train , validation and test set for this there are two strategy we can direclty use the trani_test_split from sklearn and cleverly make the train , validation and test set.
# second option we can create custom random split function to create the train , validation and test set.

train 70%,

validation 10%,

test 20%

creating a random split function which will split the dataset into train , validation and test set


In [103]:
def random_split(df, train_frac , valid_frac):
    df = df.sample(frac =1, random_state=42, reset_index(drop=True))
    train_size = int(len(df) * train_frac)
    valid_size = int(len(df) * valid_frac)
    train_df = df.iloc[:train_size]
    valid_df = df.iloc[train_size:train_size + valid_size]
    test_df = df.iloc[train_size + valid_size:]
    return train_df, valid_df, test_df


SyntaxError: positional argument follows keyword argument (3078188764.py, line 2)

# option 2 from sklearn

In [94]:
from sklearn.model_selection import train_test_split

note train test split can make two splits train and test, we will first create 70% train_df and 30% temp_test_df split, then we will further split on temp_test_df to make 
10% val_df and 20% test_df. In this way we can get splitting in the ratio 70:10:20

In [138]:
training_df, temp_test_df = train_test_split(balanced_df,test_size=0.3, random_state=42) #
val_df, test_df = train_test_split(temp_test_df, test_size=2/3, random_state = 42)


In [148]:
training_df.head()

,Label,Text
1301,0,Great to hear you are settling well. So what's...
2023,0,"I don't have anybody's number, I still haven't..."
5521,0,No. I dont want to hear anything
2695,0,All these nice new shirts and the only thing I...
3485,0,"Hello, my love! How goes that day ? I wish you..."


In [145]:
val_df.head()

,Label,Text
5028,1,Ur cash-balance is currently 500 pounds - to m...
1127,1,For taking part in our mobile survey yesterday...
5286,1,URGENT! You have won a 1 week FREE membership ...
2575,1,Congrats 2 mobile 3G Videophones R yours. call...
697,0,Good. Good job. I like entrepreneurs


In [142]:
test_df.head()

,Label,Text
2934,0,Only 2% students solved this CAT question in '...
3863,1,Free Msg: Ringtone!From: http://tms. widelive....
5428,1,Santa Calling! Would your little ones like a c...
1219,0,"Damn, can you make it tonight or do you want t..."
4827,0,I am going to sleep. I am tired of travel.


# converting these dataframe into csv files so that we can use them for training and testing and validation also

In [140]:
training_df.to_csv("./balanced_dataset_prepared/splits/train.csv", index=None)
test_df.to_csv("./balanced_dataset_prepared/splits/testing.csv", index=None)
val_df.to_csv("./balanced_dataset_prepared/splits/validation.csv", index=None)

In [141]:
print(balanced_df['Label'].value_counts())
print(training_df.shape)
print(val_df.shape)
print(test_df.shape)

Label
0    747
1    747
Name: count, dtype: int64
(1045, 2)
(149, 2)
(300, 2)
